In [ ]:
import numpy as np
import os 

In [117]:
class Individual:
    def __init__(self, individual = None):
        self.genotype = individual if individual else list()

    def fitness(self, cost_matrix) -> int:
        res = 0
        n = len(self.genotype)

        for c in range(n):
            i = self.genotype[c]
            j = self.genotype[(c+1)%n]

            res += cost_matrix[i][j]
        return res

In [211]:
class Solution:
    def __init__(self, cost_matrix, population_size, offspring_size, mutation_rate, rng):
        self.cost_matrix = cost_matrix
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.mutation_rate = mutation_rate
        self.rng = rng
    
    def get_next_city(self, city_from, remaining, greedy: bool):
        
        if greedy:
            values = self.cost_matrix[city_from]
            return min( (i for i, v in enumerate(values) if v != 0 and i in remaining), key = lambda i: values[i])
        else:
            remaining_list = list(remaining)
            return remaining_list[self.rng.integers(0, len(remaining))]

        
        
    #prob of having a random pick instead of the greedy one 
    def generate_initial_population(self, p = 0.2) -> list[Individual]:
        mapped_cities = set([i for i in range(self.cost_matrix.shape[0])])
        individuals = []

        for _ in range(self.population_size):
            remaining = mapped_cities.copy()
            current_individual = Individual()

            city_from = self.rng.choice(list(remaining))
            remaining.remove(city_from)
            current_individual.genotype.append(city_from)

            while(len(remaining) > 0 ):
                
                if self.rng.random() > p : #greedy approach, take next best
                    next_city =  self.get_next_city(city_from, remaining, True)
                else: #go for a random one
                    next_city = self.get_next_city(city_from, remaining, False)
                
                remaining.remove(next_city)
                current_individual.genotype.append(next_city)
                city_from = next_city
            
            individuals.append(current_individual)

        return individuals

    #perform inversion mutation in order to minimize changes in edges (most neighbours stay neighbours)
    def perform_inversion_mutation(self, individual: Individual) -> Individual:
        new_geno = individual.genotype[:]
        n = len(new_geno)
        
        i, j = sorted(self.rng.choice(n, size=2, replace=False))

        new_geno[i:j+1] = new_geno[i:j+1][::-1]

        return Individual(new_geno)
        
    def perform_tournament_selection(self, population: list[Individual], tau: int = 5) -> Individual:
        pool = self.rng.choice(population, size=tau, replace = False)
        return min(pool, key=lambda i: i.fitness(self.cost_matrix))

    #perform crossover
    def perform_xover(self, p1: Individual, p2: Individual) -> Individual:
        edges = {city: set() for city in p1.genotype} #initialize a set of neighbours for all the citiies
        n = len(p1.genotype)

        for parent in [p1.genotype, p2.genotype]: #scroll both parents, add in the set of a city all the neighbours that appear in both parents
            for i in range(n):
                left = parent[i - 1]
                right = parent[(i + 1) % n]
                edges[parent[i]].update([left, right]) #Add both left and right neighbour

        current = self.rng.choice(p1.genotype) #we start from a random city
        child = [current] #child is initialized with this city

        for e in edges.values(): #the city is already visited, we remove it from edges
            e.discard(current)

        while len(child) < n: #until the child contains all the cities, we proceed
            neighbours = edges[current] #get the neighbours of the current city
            
            if neighbours: #from all the neighbours cities, we select the one that has less neighbours
                next_city = min(neighbours, key = lambda c: len(edges[c]))
            else: #if the selected city has no neighbours, we select a random city from the remaining ones
                remaining = set(p1.genotype) - set(child)
                next_city = self.rng.choice(list(remaining))

            child.append(next_city) #add the city to the child and remove it from the edges

            for e in edges.values():
                e.discard(next_city)
            current = next_city

        return Individual(child)
    
    def solve(self, max_iter = 1000, p = 0.2):

        current_population = self.generate_initial_population(p)
        current_population_cp = sorted(current_population, key=lambda i: i.fitness(self.cost_matrix), reverse=True)
        print(f"Initial fitness: {   current_population_cp[0].fitness(self.cost_matrix)}")
        
        best_fitness = float('inf')
        idle_counter = 0

        for _ in range(max_iter):
            offspring = list()
            
            if idle_counter > 10:
                current_mutation_rate = min(0.9, self.mutation_rate * 1.5)
            elif idle_counter > 5:
                current_mutation_rate = min(0.7, self.mutation_rate * 1.2)
            else:
                current_mutation_rate = self.mutation_rate

            for o in range(self.offspring_size):

                if self.rng.random() < current_mutation_rate:
                    p = self.perform_tournament_selection(current_population)
                    o = self.perform_inversion_mutation(p)
                else:
                    p1 = self.perform_tournament_selection(current_population)
                    p2 = self.perform_tournament_selection(current_population)
                    o = self.perform_xover(p1,p2)

                offspring.append(o)

            # steady state
            current_population.extend(offspring)


            # survival selection
            current_population = sorted(current_population, key=lambda i: i.fitness(self.cost_matrix))
            current_fitness = current_population[0].fitness(self.cost_matrix)
            
            if current_fitness < best_fitness:
                best_fitness = current_fitness
                idle_counter = 0
            else:
                idle_counter += 1


            current_population = current_population[:self.population_size]
            

        return best_fitness


 
        


In [ ]:
rng = np.random.default_rng(seed = 42)

files = sorted(os.listdir("./data"), key=lambda x: (
    x.split('_')[1], 
    int(x.split('_')[2].split('.')[0])
))

for file in files:
    cost_matrix = np.load(os.path.join("./data", file))
    n = cost_matrix.shape[0]

    population_size = max(50, n) 
    offspring_size = population_size // 2

    print(f"------------- FILE {file} -------------------")
    solver = Solution(cost_matrix, population_size, offspring_size, 0.5, rng)
    best_fitness = solver.solve(max_iter = 50, p = 0.1)
    print(f"Final fitness: {best_fitness}\n")


------------- FILE problem_g_10.npy -------------------
Initial fitness: 1846.6600654629146
Final fitness: 1497.6636482252907

------------- FILE problem_g_20.npy -------------------
Initial fitness: 2292.3076244771896
Final fitness: 1755.5146770830045

------------- FILE problem_g_50.npy -------------------
Initial fitness: 3750.120420657874
Final fitness: 2891.441961190176

------------- FILE problem_g_100.npy -------------------
Initial fitness: 5305.351126896666
Final fitness: 4266.213971624785

------------- FILE problem_g_200.npy -------------------
Initial fitness: 7859.463947106634
Final fitness: 6043.799149999541

------------- FILE problem_g_500.npy -------------------
Initial fitness: 10896.133919712722
Final fitness: 9582.614432337794

------------- FILE problem_g_1000.npy -------------------
Initial fitness: 15110.890537844563
Final fitness: 14025.740186735275

------------- FILE problem_r1_10.npy -------------------
Initial fitness: 296.8879913574428
Final fitness: 184.27